In [1]:
import pandas as pd

In [10]:
data=pd.read_csv('/home/shayantan/Desktop/NOTEEVENTS.csv')

/home/shayantan/anaconda3/envs/tensorflow/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
diag = pd.read_csv('/home/shayantan/Desktop/DIAGNOSES_ICD.csv')


In [13]:
diag.head(5)

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,58281
3,1300,109,172335,4.0,5855
4,1301,109,172335,5.0,4254


In [16]:
data.CATEGORY.value_counts()

Nursing/other        822497
Radiology            522279
Nursing              223556
ECG                  209051
Physician            141624
Discharge summary     59652
Echo                  45794
Respiratory           31739
Nutrition              9418
General                8301
Rehab Services         5431
Social Work            2670
Case Management         967
Pharmacy                103
Consult                  98
Name: CATEGORY, dtype: int64

In [17]:
text=data[data.CATEGORY=='Discharge summary']

In [18]:
data_key=pd.merge(diag[['SUBJECT_ID','HADM_ID','ICD9_CODE']],text[['SUBJECT_ID','HADM_ID','CATEGORY','TEXT']],on=['SUBJECT_ID','HADM_ID'],how='left')

In [19]:
data_key.head(5)

,SUBJECT_ID,HADM_ID,ICD9_CODE,CATEGORY,TEXT
0,109,172335,40301,Discharge summary,Admission Date: [**2141-9-18**] ...
1,109,172335,486,Discharge summary,Admission Date: [**2141-9-18**] ...
2,109,172335,58281,Discharge summary,Admission Date: [**2141-9-18**] ...
3,109,172335,5855,Discharge summary,Admission Date: [**2141-9-18**] ...
4,109,172335,4254,Discharge summary,Admission Date: [**2141-9-18**] ...


In [24]:
data_sepsis=data_key[(data_key.ICD9_CODE=='99591') | (data_key.ICD9_CODE=='78552')]

In [26]:
data_sepsis.shape

(4182, 5)

In [27]:
def preprocess_text(df):
    # This function preprocesses the text by filling not a number and replacing new lines ('\n') and carriage returns ('\r')
    df.TEXT = df.TEXT.fillna(' ')
    df.TEXT =df.TEXT.str.replace('\n',' ')
    df.TEXT =df.TEXT.str.replace('\r',' ')
    df.TEXT = df.TEXT.str.replace('\d+', '')
    df.TEXT=df.TEXT.str.replace('\[\*\*[^\]]*\*\*\]', '')
    df.TEXT=df.TEXT.str.replace('<[^>]*>','')
    df.TEXT=df.TEXT.str.lower()
    df.TEXT=df.TEXT.str.replace(r'[^a-z0-9]+', ' ')
    df.TEXT=df.TEXT.str.replace(r'\b\w\b','').str.replace(r'\s+', ' ')
    df.TEXT=df.TEXT.str.replace(" *\\b[[:alpha:]]{1,2}\\b *", '')

    return df

In [36]:
data_cleaned_sepsis=preprocess_text(data_sepsis)

/home/shayantan/anaconda3/envs/tensorflow/lib/python3.6/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


#### Each note form the data_cleaned_sepsis dataframe was then fed to the EHR-phenolyzer which then outputted a list of phenotypes and causal genes. The predicted list of only sepsis causing genes was set aside for further analysis